In [2]:
import pandas as pd

# df1 = pd.read_csv("./data/0401/nbroad-v2.csv")
# df2 = pd.read_csv("./data/0401/gemma_suppl_rewrite.csv")

# df2['rewrite_prompt'] = df2['rewrite_prompts']

# df = pd.concat([df1[['rewrite_prompt']], df2[['rewrite_prompt']]])

df = pd.read_csv("./data/0304/seed_final.csv")

df.iloc[0]['rewrite_prompt']

'Rewrite the essay as though it were a news article'

In [3]:
# encode
from sentence_transformers import SentenceTransformer

t5_base = SentenceTransformer('sentence-t5-base')

embeddings = t5_base.encode(df['rewrite_prompt'].tolist()).tolist()

embeddings

[[-0.05879785120487213,
  0.028493080288171768,
  0.012155168689787388,
  0.04490673169493675,
  0.007927936501801014,
  0.004745964892208576,
  -0.014194338582456112,
  0.05964633449912071,
  -0.0030585303902626038,
  -0.022772815078496933,
  0.05061252787709236,
  -0.021743169054389,
  0.050397004932165146,
  0.010818730108439922,
  0.04943554848432541,
  0.04440370202064514,
  -0.017186686396598816,
  -0.012703528627753258,
  0.06275871396064758,
  0.043758463114500046,
  0.0534793846309185,
  -0.028260504826903343,
  -0.016774769872426987,
  0.001845548045821488,
  0.013671323657035828,
  0.037253450602293015,
  0.058735109865665436,
  -0.030192619189620018,
  -0.010144341737031937,
  0.008258870802819729,
  0.06316323578357697,
  -0.036612626165151596,
  -0.026206493377685547,
  0.06645150482654572,
  -0.02679927833378315,
  0.009953933767974377,
  -0.03137040510773659,
  -0.062120869755744934,
  -0.0490645132958889,
  0.00034580149804241955,
  0.005809876136481762,
  -0.013908186

In [5]:
from sklearn.cluster import KMeans
import numpy as np

def diverse_subset(vectors, k):
    # Convert vectors to numpy array
    X = np.array(vectors)
    
    # Perform K-means clustering
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X)
    
    # Get the cluster labels
    cluster_labels = kmeans.labels_
    
    # Find the centroids of each cluster
    centroids = kmeans.cluster_centers_
    
    # Initialize a list to store the diverse subset
    diverse_subset = []
    
    # Select one vector from each cluster to include in the diverse subset
    for i in range(k):
        cluster_indices = np.where(cluster_labels == i)[0]
        if len(cluster_indices) > 0:
            # Choose a representative vector from the cluster (e.g., centroid)
            representative_index = np.argmin(np.linalg.norm(X[cluster_indices] - centroids[i], axis=1))
            diverse_subset.append(cluster_indices[representative_index])
    
    return diverse_subset

subset = diverse_subset(embeddings, 3)
print("Diverse subset:", len(subset))

Diverse subset: 3


/opt/miniconda3/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [6]:
df_shuffle = df.iloc[subset].sample(frac=1).reset_index(drop=True)
df_shuffle.to_csv("./data/0401/nbroad-v2-gemma_suppl_rewrite.csv", index=False, columns=['rewrite_prompt'])

df_shuffle.iloc[0]['rewrite_prompt']

'Rewrite the sentence in a more concise and engaging way.'

# Build predict datasets

In [7]:
import random

random.seed(42)

df_prompts = pd.read_csv("./data/0401//nbroad-v2-gemma_suppl_rewrite.csv")
df_text = pd.read_csv("./data/0401/150_suppl_original_text.csv")

prompts = df_prompts.iloc[:200]['rewrite_prompt'].tolist()

df_rows = []
for p in prompts:
    texts = df_text.sample(frac=0.2)['original_text'].to_list()[:30]
    for t in texts:
        df_rows.append({'rewrite_prompt': p, 'original_text': t})

df = pd.DataFrame(df_rows)
df.to_csv("./data/0401/200px300t_data_predict.csv", index=False)